In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os, random
import numpy as np

from torchvision import transforms, datasets


"""
[주의!]
상단 코드는 수정하지 마세요
"""
###################################################################


class MyModel(nn.Module):
############# 해당 부분을 구현하시면 됩니다#########################
    def __init__(self, input_dim, hidden_dim, layers):
        super().__init__()        
        
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x


###################### 구현 부분 끝  ##############################
###################################################################
"""
[주의!]
하단 코드는 수정하지 마세요
"""


##SEED값 고정
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
SEED = 123
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)


if DEVICE.type == "cuda":
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
 


sentence = ("There are many variations of passages of Lorem Ipsum available"
            "but the majority have suffered alteration in some form, by injected humour"
            "randomised words which don't look even slightly believable."
            "If you are going to use a passage of Lorem Ipsum"
            "you need to be sure there isn't anything embarrassing hidden in the middle of text.")

# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10
learning_rate = 0.001

# data setting
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot).to(DEVICE)
Y = torch.LongTensor(y_data).to(DEVICE)

model = MyModel(dic_size, hidden_size, 2).to(DEVICE)

# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), learning_rate)


# start training
for i in range(1, 601):
    optimizer.zero_grad()
    outputs = model(X).to(DEVICE)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]
    if i % 100 == 0:
        print("Output Result: ", predict_str)
        print("Loss value: \n", loss.item())


Output Result:     e  o  ea e  a ea e   ea  ea ea e  a  ea e     oo   e ea e  e  e   a  ea e  ea e  a       o e    ea  ea ea      e  ea ea e   e  a e  e  e e     ea e  ea ea ea    ea e       o    ea ea ea   e  e  ea ea e    ea ea ea ea ea ea e  a ea e     oo          o ea ea e  a  a ea  e  o  e    ea e e  ea   ea e    o  o  a  e e  a ea e 
Loss value: 
 2.759727954864502
Output Result:  iere  nl thsy hareanion  af Larsage  mf sorem Ihsumeare lasle ue toe massrea  hare tom  re  hneed de n tn tom  norem bl hone e d ho uaree den s d ahres shedh bon e toru   en aledhi l hade nesladaa sor mre moing torsueda lassage of sorem Ihsumeau re   io su tore there  nn t tny hen  tnearyansan  hedde  in the maddle hf sadeh
Loss value: 
 1.8897075653076172
Output Result:  here  se gany pareadion  af Lassage  af Lorem Ipsumyavailablabut ihe madsreth hare sumfereg alieraieon tn tome forem by inge ted humeurragdonised hords Ihidh don't look eren slightly pedeevabledIf you mre going to use a passage of sore